In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

REMOTE_RTS_DATA = "/media/sinergia/RTS/"
REMOTE_VIDEOS = '/mnt/rts/'

LOCAL_RTS_DATA = "/media/data/rts/"
METADATA = LOCAL_RTS_DATA + 'metadata'
LOCAL_VIDEOS = LOCAL_RTS_DATA + 'archive'

OUTDIR = 'data'


In [ ]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple
from pathlib import Path

In [ ]:
# LOCAL imports
import rts

LOG = rts.utils.get_logger()

In [ ]:
# vidx = rts.ingest.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.ingest.read_all_metadata(METADATA, vidx)
# rts.ingest.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.ingest.load_metadata_hdf5(METADATA, 'rts_metadata')

In [ ]:
client = rts.db.fulltext.get_client()
midx = rts.db.fulltext.get_or_create_media_index(client)

In [ ]:
# Fulltext Index from HDF5
# midx.delete_all_documents()
rts.db.fulltext.index_media(df, midx)
# Wait 30 sec for full indexation
midx.get_stats().number_of_documents  # 522103

In [ ]:
#TODO get unique geo, people, for the whole media

In [ ]:
a = df.groupby('collection')['collection'].count().sort_values(ascending=False)
a.to_csv(os.path.join(OUTDIR, 'collections.csv'))

In [ ]:
a = df.groupby('contentType')['contentType'].count().sort_values(ascending=False)
a.to_csv(os.path.join(OUTDIR, 'contentType.csv'))

In [ ]:
r = rts.io.archive.transcribe_media(os.path.join(OUTDIR, 'myaudio.m4a'))

In [ ]:
df.publishedBy.apply(lambda x: len(x) if x else 0).unique()

In [ ]:
input_video_folder = os.path.join(REMOTE_VIDEOS, '0', '0', '0', 'ZB001000')
# input_video_folder = os.path.join(VIDEOS, '0', '0', 'ZB002000')
# v, a = rts.io.archive.get_raw_video_audio_parts(input_video_folder)

# '/'.join(input_video_folder.split('/')[-4:])
remuxed = rts.io.archive.create_optimized_media(input_video_folder, LOCAL_VIDEOS, force=True)
print(remuxed)


In [ ]:
%ls {remux_folder}

In [87]:
scenes = rts.io.archive.extract_scenes(
    remuxed.get('media_folder'),
    remuxed.get('video'))

INFO:pyscenedetect:Downscale factor set to 3, effective resolution: 256 x 192
INFO:pyscenedetect:Detecting scenes...


[autoreload of rts.io.archive failed: Traceback (most recent call last):
  File "/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/home/kirell/.cache/pypoetry/virtualenvs/rts-506MfBOj-py3.10/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 293, in update_function
    setattr(old, name, getattr(new, name))
ValueError: create_optimized_media() requires a code object with 0 free vars, not 1
]
[autoreload of rts.io.media failed: Traceback (most recent call la

In [ ]:
paths = rts.io.media.save_scenes_images(scenes, remuxed['video'], remuxed['media_folder'])

In [ ]:
images_folder = Path(remuxed['media_folder']) / 'images'
os.makedirs(str(images_folder), exist_ok=True)

In [ ]:
rts.utils.obj_to_json(r, os.path.join(OUTDIR, 'transcript.json'))

In [ ]:
(t[1] - t[0]).get_seconds()


In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.to_mp3(a, os.path.join(OUTDIR, 'audio.mp3'))

In [ ]:
rts.io.media.extract_audio(a, os.path.join(OUTDIR, 'myaudio'))

In [ ]:
import av
import av.datasets


video_part = os.path.join(VIDEOS, '0', '0', 'ZB001000')
# video_part = os.path.join(VIDEOS, '0', '0', 'ZB002000')
v, a = rts.io.archive.get_raw_video_audio_parts(video_part)
out_path = os.path.join(OUTDIR, 'thumbs')

content = v

print(content)

with av.open(content) as container:
    # Signal that we only want to look at keyframes.
    stream = container.streams.video[0]
    stream.codec_context.skip_frame = "NONKEY"

    # print(stream.codec_context.height)
    for i, frame in enumerate(container.decode(stream)):
        if i % 4 != 0:
            continue


    #     print(frame)
    #     print(frame.time, frame.time_base)
        # We use `frame.pts` as `frame.index` won't make must sense with the `skip_frame`.
        frame.to_image().save(
            os.path.join(out_path, f"f{frame.pts:04d}.jpg"),
            quality=80,
        )

In [ ]:
thumbs_dir = os.path.join(OUTDIR, 'thumbs')
outfast = './notebook/fastdup'
fastdup_data = './notebook/out'

In [ ]:
import fastdup

In [ ]:

fastdup.run(input_dir=thumbs_dir, work_dir=fastdup_data, nearest_neighbors_k=8, turi_param='ccthreshold=0.90')    #main running function.
# fastdup.create_duplicates_gallery('out/similarity.csv', save_path=outfast)     #create a visual gallery of found duplicates
# fastdup.create_outliers_gallery('out/outliers.csv',   save_path=outfast)       #create a visual gallery of anomalies
# fastdup.create_components_gallery(fastdup_data, save_path=outfast)                    #create visualiaiton of connected components
# fastdup.create_stats_gallery('out', save_path=outfast, metric='blur')          #create visualization of images stastics (for example blur)
# fastdup.create_similarity_gallery('out', save_path=outfast,get_label_func=lambda x: x.split('/')[-2])     #create visualization of top_k similar images assuming data have labels which are in the folder name
# fastdup.create_aspect_ratio_gallery('out', save_path=outfast)

In [ ]:
config = fastdup.load_config(fastdup_data)
df = fastdup.galleries.do_find_top_components(work_dir=fastdup_data, input_dir=config['input_dir'], kwargs={})

In [ ]:
df.len.sum()

In [ ]:
df